<a href="https://colab.research.google.com/github/Pasq98/Tesi/blob/main/PrimoEsperimentoDeep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Librerie Deep Learning
#Deep learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import InputLayer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys, getopt, pickle, csv, sklearn, re
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os
os.environ['KERAS_BACKEND']='theano'
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Embedding, Reshape, Input, Concatenate
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Model,Sequential
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, optimizers
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SpatialDropout1D
from keras import initializers, regularizers, constraints


from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from keras.preprocessing.sequence import pad_sequences

In [2]:
#@title Load GLOVE vectors
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("/content/gensim_glove_vectors.txt", binary=False)

In [3]:
#@title Evaluate deep model
#EVALUATE DEEP MODEL
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (classification_report,
                             confusion_matrix,
                             roc_auc_score)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
  
#report = classification_report(y_test, y_pred)
#print(report)
  
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title("Confusion matrix (non-normalized))")
    plt.ylabel("Actual label")
    plt.xlabel("Predicted label")

In [4]:
#@title Load Dataset
def load_Cybertroll(): #Carico il dataset e ritorno il testo e i label
  CBT = pd.read_json('/content/Dataset for Detection of Cyber-Trolls.json', lines= True)
  #np.random.seed(10)
  remove_n = 1
  drop_indices = np.random.choice(CBT.index, remove_n, replace=False)
  CBT = CBT.drop(drop_indices)
  

  CBT["label"] = CBT.annotation.apply(lambda x: x.get('label'))
  CBT["label"] = CBT.label.apply(lambda x: x[0])

  x = CBT['content']
  label = CBT['label']

  return x, label

def load_Formsprings():
    FS = pd.read_csv('/content/formspringPicke.csv')
    FS = FS[FS['text'].notnull()] #delete NaN
    #FS['text']=FS['text'].apply(str)
    x = FS['text']
    label = FS['label']
    return x,label

def load_Tweets(): 
  tweets=pd.read_csv('/content/Tweets.csv')
  tweets_df=tweets.drop(tweets[tweets['airline_sentiment_confidence']<0.5].index,axis=0)
  tweets_df['text'] = tweets_df['text'].str.replace('@AmericanAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@USAirways', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@SouthwestAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@JetBlue', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@VirginAmerica', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@united', '')#Rimuovo hastag iniziali

  #Rimuovo tweet con sentimento neutro
  tweets_df = tweets_df.loc[tweets_df["airline_sentiment"] != 'neutral']
  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('positive', '1')#Rimuovo hastag iniziali
  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('negative', '0')#Rimuovo hastag iniziali

  x=tweets_df['text'] #testo con caratteri speciali
  label=tweets_df['airline_sentiment']
  return x, label

def load_ACLL():
  acll=pd.read_csv('/content/ACLLTrain.csv')
  x = acll['Review']
  label = acll['Rating']
  return x, label


In [5]:
#@title Standard preprocessing
def standard_preprocessing(text):
  clean_text = text.str.lower()
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  # Remove trailing spaces
  clean_text = clean_text.str.strip()
  # Remove Punctuations
  clean_text = clean_text.str.replace('[^\w\s]',' ')
  # Remove <br>
  clean_text = clean_text.str.replace('br', '')
  # Remove extra space in between words
  clean_text = clean_text.str.replace(' +', ' ')
  clean_text = clean_text.str.replace('\n', ' ')# per pad sequence

  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  #stop word remove 
  stop = stopwords.words('english')
  clean_text = clean_text.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
  #token
  #clean_text = clean_text.apply(nltk.word_tokenize) #NON FARLO CON EMBEDDING
  return clean_text

In [ ]:
#@title Deep Model

def blstm(inp_dim,vocab_size, embed_size, emb_weight):   
    model = Sequential()
    model.add(Embedding(vocab_size, embed_size, weights=[emb_weight] ,input_length=inp_dim, trainable=True))
    model.add(Dropout(0.25))
    model.add(Bidirectional(LSTM(embed_size)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def cnn_model(inp_dim, vocab_size, embed_size, emb_weight):
  model = Sequential()
  model.add(Embedding(vocab_size, embed_size, weights=[emb_weight],input_length=inp_dim, trainable=True))
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

from keras.layers import Activation, MaxPooling1D, BatchNormalization, GRU, Bidirectional, SpatialDropout1D, Conv1D
!pip install attention
from attention import Attention
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
def BiLSTMAttention(inp_dim, vocab_size, embed_size, emb_weight):
  attention_model = Sequential()
  attention_model.add(Embedding(vocab_size, embed_size, weights=[emb_weight] ,input_length=inp_dim, trainable=True))
  attention_model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  attention_model.add(MaxPooling1D(pool_size=2))
  attention_model.add(Dropout(0.5))
  attention_model.add(Bidirectional(LSTM(300, return_sequences=True)))
  attention_model.add(Attention())
  attention_model.add(Dense(1, activation='sigmoid'))
  nadam = optimizers.Nadam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
  attention_model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
  return attention_model

In [7]:
#@title Get embedding matrix

def get_embedded_matrix(size_vocab):
  # create a weight matrix for words in training docs
  embedding_matrix = np.zeros((size_vocab, 50))

  for word, i in tokenizer.word_index.items():
    try:
      embedding_vector = glove_model.get_vector(word)
      embedding_matrix[i] = embedding_vector
    except KeyError:
      embedding_vector = 0
      embedding_matrix[i] = embedding_vector
  return embedding_matrix


In [ ]:
#@title TEST DEEP LEARNING, STANDARD E EMBEDDING
x, label = load_Cybertroll()
x = standard_preprocessing(x)
#print(label)

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x))
MAX_LEN = 50 
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x, MAX_LEN)#MAX LEN dovrebbe essere il numero di massimo di parole in una frase

label = [int(lab) for lab in label] #Necessario per piu dataset

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print("Parole del dizionario: ",size_of_vocabulary)

embedding_matrix = get_embedded_matrix(size_of_vocabulary)

estimator = KerasClassifier(build_fn=lambda: Didio(MAX_LEN,size_of_vocabulary,50,0.01), epochs=10, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
print(x.shape)
results = cross_val_predict(estimator, x, label, cv=kfold, n_jobs=-1)
print(classification_report(label, results))







In [ ]:
#@title Stemming
def standardStemming(text):
  stemmer= PorterStemmer()

  clean_text = text.str.lower()
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  # Remove trailing spaces
  clean_text = clean_text.str.strip()
  # Remove Punctuations
  clean_text = clean_text.str.replace('[^\w\s]',' ')
  # Remove <br>
  clean_text = clean_text.str.replace('br', '')
  # Remove extra space in between words
  clean_text = clean_text.str.replace(' +', ' ')
  clean_text = clean_text.str.replace('\n', ' ')# per pad sequence
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  #stop word remove 
  stop = stopwords.words('english')
  clean_text = clean_text.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
  #Stemming
  clean_text = clean_text.apply(lambda x: " ".join(stemmer.stem(x) for x in x.split() ))
 
  return clean_text

In [ ]:
#@title TEST STANDARD+STEMMING+EMBEDDING
x, label = load_ACLL()
x = standardStemming(x)
#print(label)

#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x))
MAX_LEN = 50 
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x, MAX_LEN)#MAX LEN dovrebbe essere il numero di massimo di parole in una frase

label = [int(lab) for lab in label] #Necessario per piu dataset

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print("Parole del dizionario: ",size_of_vocabulary)

embedding_matrix = get_embedded_matrix(size_of_vocabulary)


estimator = KerasClassifier(build_fn=lambda: blstm(MAX_LEN,size_of_vocabulary,50,0.01), epochs=10, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
print(x.shape)
results = cross_val_predict(estimator, x, label, cv=kfold, n_jobs=-1)
print(classification_report(label, results))

In [ ]:
#@title Deep Model TF-idf

def blstm(len):   
    model = Sequential()  
    model.add(Embedding(input_dim=len,output_dim=300))
    model.add(Dropout(0.55))
    model.add(Bidirectional(LSTM(300)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

def cnn_model():
  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model

from keras.layers import Activation, MaxPooling1D, BatchNormalization, GRU, Bidirectional, SpatialDropout1D, Conv1D
!pip install attention
from attention import Attention
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
def BiLSTMAttention(len):
  attention_model = Sequential()
  attention_model.add(Embedding(input_dim=len,output_dim=300))
  attention_model.add(Bidirectional(LSTM(300, return_sequences=True)))
  attention_model.add(Attention())
  attention_model.add(Dense(1, activation='sigmoid'))
  nadam = optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
  return attention_model

In [ ]:
#@title Test TF IDF
x, label = load_Cybertroll() 
x = standard_preprocessing(x)#Preprocessing
print(x.shape)

#lower_sentences = [" ".join(j) for j in x]

#Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (1, 2), max_features = 50)# ngram_range of (1, 1) means only unigrams,(1, 2) means unigrams and bigrams,and(2, 2) means only bigrams
X = vectorizer.fit_transform(x).toarray() 
tf_len = len(vectorizer.vocabulary_)
print(tf_len)
#X = X[..., None]
#label = np.array(label)
#label = label[..., np.newaxis]

## Making predictions on our model
estimator = KerasClassifier(build_fn=lambda: blstm(tf_len), epochs=10, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_predict(estimator, X, label, cv=kfold)
print(classification_report(label, results))
